In [1]:
import copy
import math
from math import exp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_curve,
    precision_recall_curve,
    auc as calc_auc,
)


In [54]:
path_to_data = '../public_data/mixmhcpred/TableS2.txt'
# ignore the first row (header)
df = pd.read_csv(path_to_data, sep='\t', skiprows=1)
X = df.drop('Allele', axis=1)
y = list(df['Allele'].values)

In [55]:
X = [sequence.ljust(14, "-") for sequence in X['Peptide']]

In [67]:

amino_acids = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]

chars_peptide = amino_acids + ["-"]

peptide_encoding_size = len(chars_peptide)
print(''.join(chars_peptide))

# create a mapping from amino-acids to integers
stoi_peptide = { ch:i for i,ch in enumerate(chars_peptide) }
itos_peptide = { i:ch for i,ch in enumerate(chars_peptide) }
encode_peptide = lambda s: [stoi_peptide[c] for c in s] # encoder: take a string, output a list of integers
decode_peptide = lambda l: ''.join([itos_peptide[i] for i in l]) # decoder: take a list of integers, output a string

print(encode_peptide("AAAHTHRY------"))
print(decode_peptide(encode_peptide("AAAHTHRY------")))


ARNDCQEGHILKMFPSTWYV-
[0, 0, 0, 8, 16, 8, 1, 18, 20, 20, 20, 20, 20, 20]
AAAHTHRY------


In [57]:
X = np.array([encode_peptide(sequence) for sequence in X])

In [64]:
# now encode HLAs as integers
hlas = list(set(y))
hla_size = len(hlas)
# create a mapping from amino-acids to integers
stoi_hla = { ch:i for i,ch in enumerate(hlas) }
itos_hla = { i:ch for i,ch in enumerate(hlas) }
encode_hla = lambda s: [stoi_hla[c] for c in s] # encoder: take a string, output a list of integers
decode_hla = lambda l: ''.join([itos_hla[i] for i in l]) # decoder: take a list of integers, output a string
y = np.array(encode_hla(y))

In [62]:

# let's now encode the entire dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(X, dtype=torch.long)
print(data.shape, data.dtype)

ydata = torch.tensor(y, dtype = torch.float)

# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
train_y = ydata[:n].long()
val_y = ydata[n:].long()


torch.Size([384070, 14]) torch.int64


In [97]:
#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils.data import DataLoader, TensorDataset
#import numpy as np


# Define the Transformer-based model architecture.
#class TransformerModel(nn.Module):
#    def __init__(self, input_dim, output_dim, nhead, num_encoder_layers, hidden_dim, dropout):
#        super(TransformerModel, self).__init__()
#        self.embedding = nn.Embedding(input_dim, hidden_dim)
#        self.transformer = nn.Transformer(
#            d_model=hidden_dim,
#            nhead=nhead,
#            num_encoder_layers=num_encoder_layers,
#            dim_feedforward=hidden_dim * 4,
#            dropout=dropout,
#            batch_first=True,
#        )
#        self.fc = nn.Linear(hidden_dim, output_dim)
#        
#    def forward(self, x):
#        x = self.embedding(x)
#        x = self.transformer(x)
#        x = x.mean(dim=1)  # Global average pooling
#        x = self.fc(x)
#        return x

# Define hyperparameters and create the model instance.
#input_dim = 20+1  # Example: dimension of one-hot encoding for amino acids
#output_dim = 161  # Number of classes
#nhead = 1  # Number of attention heads in the transformer
#num_encoder_layers = 1  # Number of encoder layers in the transformer
#hidden_dim = 9  # Hidden dimension of the transformer
#dropout = 0.2

#model = TransformerModel(input_dim, output_dim, nhead, num_encoder_layers, hidden_dim, dropout)
#print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters')

In [101]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
max_iters = 1500
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 18
n_head = 2
n_layer = 2
dropout = 0.25
pmhc_length = 14
# ------------



# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ypred = train_y if split == 'train' else val_y
   
    ix = torch.randint(len(data), (batch_size,))
    x = torch.stack([data[i] for i in ix])
    y = torch.stack([ypred[i] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
       
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple pMHC model
class pMHCLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(peptide_encoding_size, n_embd)
        self.position_embedding_table = nn.Embedding(pmhc_length, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, hla_size)


    def forward(self, idx, targets=None):
        B, T = idx.shape
       
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
       
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T)
               
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            print(targets.shape)
            print(logits.shape)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss


In [104]:
model = pMHCLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e3, 'K parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


11.063 K parameters
torch.Size([32])
torch.Size([32, 14, 119])


RuntimeError: shape '[448]' is invalid for input of size 32

In [108]:
import random
y_fake_data = torch.LongTensor([random.randint(0, 99) for _ in range(50)])  # Fake labels

In [109]:
y_fake_data

tensor([34, 77, 74,  5, 38, 98, 80, 28, 25, 72, 83,  0, 29, 67, 74, 80, 44, 33,
        65, 13,  0, 86, 68, 98, 49, 53, 22, 85, 16, 33, 82, 13, 21, 59,  4, 39,
        93, 95, 98, 58, 93, 74, 91, 12, 27, 65, 66, 96, 62, 79])